# Training a sequence tagger for Slot Filling

<a target="_blank" href="https://recognai.github.io/biome-text/v3.2.0/documentation/tutorials/2-Training_a_sequence_tagger_for_Slot_Filling.html"><img class="icon" src="https://recognai.github.io/biome-text/v3.2.0/assets/img/biome-isotype.svg" width=24 /></a>
[View on recogn.ai](https://recognai.github.io/biome-text/v3.2.0/documentation/tutorials/2-Training_a_sequence_tagger_for_Slot_Filling.html)
    
<a target="_blank" href="https://colab.research.google.com/github/recognai/biome-text/blob/v3.2.0/docs/docs/documentation/tutorials/2-Training_a_sequence_tagger_for_Slot_Filling.ipynb"><img class="icon" src="https://www.tensorflow.org/images/colab_logo_32px.png" width=24 /></a>
[Run in Google Colab](https://colab.research.google.com/github/recognai/biome-text/blob/v3.2.0/docs/docs/documentation/tutorials/2-Training_a_sequence_tagger_for_Slot_Filling.ipynb)
        
<a target="_blank" href="https://github.com/recognai/biome-text/blob/v3.2.0/docs/docs/documentation/tutorials/2-Training_a_sequence_tagger_for_Slot_Filling.ipynb"><img class="icon" src="https://github.githubassets.com/images/modules/logos_page/GitHub-Mark.png" width=24 /></a>
[View source on GitHub](https://github.com/recognai/biome-text/blob/v3.2.0/docs/docs/documentation/tutorials/2-Training_a_sequence_tagger_for_Slot_Filling.ipynb)

When running this tutorial in Google Colab, make sure to install *biome.text* first:

In [ ]:
!pip install -U pip
!pip install -U biome-text
exit(0)  # Force restart of the runtime

*If* you want to log your runs with [WandB](https://wandb.ai/home), don't forget to install its client and log in.

In [ ]:
!pip install wandb
!wandb login

## Introduction

In this tutorial we will train a sequence tagger for filling slots in spoken requests.
The goal is to look for specific pieces of information in the request and tag the corresponding tokens accordingly. 
The requests will include several intents, from getting weather information to adding a song to a playlist, each requiring its own set of slots.
Therefore, slot filling often goes hand in hand with intent classification.
In this tutorial, however, we will only focus on the slot filling task.

Slot filling is closely related to [Named-entity recognition (NER)](https://en.wikipedia.org/wiki/Named-entity_recognition) and the model of this tutorial can also be used to train a NER system.

In this tutorial we will use the [SNIPS data set](https://github.com/snipsco/nlu-benchmark/tree/master/2017-06-custom-intent-engines) adapted by [Su Zhu](https://github.com/sz128/slot_filling_and_intent_detection_of_SLU/tree/master/data/snips) and our simple [data preparation notebook](https://biome-tutorials-data.s3-eu-west-1.amazonaws.com/token_classifier/data_prep.ipynb).

### Imports

Let us first import all the stuff we need for this tutorial.

In [ ]:
from biome.text import Pipeline, Dataset, PipelineConfiguration, VocabularyConfiguration, Trainer
from biome.text.configuration import FeaturesConfiguration, WordFeatures, CharFeatures
from biome.text.modules.configuration import Seq2SeqEncoderConfiguration
from biome.text.modules.heads import TokenClassificationConfiguration

## Explore the data

Let's take a look at the data before starting with the configuration of our pipeline.
For this we create a `Dataset` instance providing a path to our downloaded data.

In [ ]:
!curl -O https://biome-tutorials-data.s3-eu-west-1.amazonaws.com/token_classifier/train.json
!curl -O https://biome-tutorials-data.s3-eu-west-1.amazonaws.com/token_classifier/valid.json
!curl -O https://biome-tutorials-data.s3-eu-west-1.amazonaws.com/token_classifier/test.json

In [ ]:
train_ds = Dataset.from_json("train.json")
valid_ds = Dataset.from_json("valid.json")
test_ds = Dataset.from_json("test.json")

The [Dataset](https://recognai.github.io/biome-text/v3.2.0/api/biome/text/dataset.html#dataset) class is a very thin wrapper around HuggingFace's awesome [datasets.Dataset](https://huggingface.co/docs/datasets/master/package_reference/main_classes.html#datasets.Dataset).
Most of HuggingFace's `Dataset` API is exposed and you can checkout their nice [documentation](https://huggingface.co/docs/datasets/master/processing.html) on how to work with data in a `Dataset`. For example, let's quickly check the size of our training data and print the first 10 examples as a pandas DataFrame:

In [ ]:
print("Training data size:", len(train_ds))

train_ds.head()

As we can see we have two relevant columns for our task: *text* and *labels*. 
The *intent* column will be ignored in this tutorial. 

The text input already comes pre-tokenized as a list of strings and each token in the *text* column has a label/tag in the *labels* column, this means that both list always have the same length.
The labels are given in the [BIO tagging scheme](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)), which is widely used in Slot Filling/NER systems.

We can quickly check how many different labels there are in our dataset:

In [ ]:
labels = {tag[2:] for tags in train_ds["labels"] for tag in tags if tag != "O"}
print("number of lables:", len(labels))
labels

Since the the [TaskHead](https://recognai.github.io/biome-text/v3.2.0/api/biome/text/modules/heads/task_head.html#taskhead) of our model (the [TokenClassification](https://recognai.github.io/biome-text/v3.2.0/api/biome/text/modules/heads/token_classification.html#tokenclassification) head) expects a *text* and a *tags* column to be present in the Dataset, we need to rename the *labels* column:

In [ ]:
for ds in [train_ds, valid_ds, test_ds]:
     ds.rename_column_("labels", "tags")

::: tip Tip

The [TokenClassification](https://recognai.github.io/biome-text/v3.2.0/api/biome/text/modules/heads/token_classification.html#tokenclassification) head also supports a *entities* column instead of a *tags* column, in which case the entities have to be a list of python dictionaries with a `start`, `end` and `label` key that correspond to the char indexes and the label of the entity, respectively.

:::

## Configure your *biome.text* Pipeline

A typical [Pipeline](https://recognai.github.io/biome-text/v3.2.0/api/biome/text/pipeline.html#pipeline) consists of tokenizing the input, extracting features, applying a language encoding (optionally) and executing a task-specific head in the end.
After training a pipeline, you can use it to make predictions

A *biome.text* pipeline has the following main components:

```yaml
name: # a descriptive name of your pipeline

tokenizer: # how to tokenize the input

features: # input features of the model

encoder: # the language encoder

head: # your task configuration

```

See the [Configuration section](https://recognai.github.io/biome-text/v3.2.0/documentation/user-guides/2-configuration.html) for a detailed description of how these main components can be configured.

In this tutorial we will create a [PipelineConfiguration](https://recognai.github.io/biome-text/v3.2.0/api/biome/text/configuration.html#pipelineconfiguration) programmatically, and use it to initialize the [Pipeline](https://recognai.github.io/biome-text/v3.2.0/api/biome/text/pipeline.html#pipeline).
You can also create your pipelines by providing a [python dictionary](https://recognai.github.io/biome-text/v3.2.0/api/biome/text/pipeline.html#from-config) (see the text classification [tutorial](https://recognai.github.io/biome-text/v3.2.0/documentation/tutorials/1-Training_a_text_classifier.html)), a YAML [configuration file](https://recognai.github.io/biome-text/v3.2.0/api/biome/text/pipeline.html#from-yaml) or a [pretrained model](https://recognai.github.io/biome-text/v3.2.0/api/biome/text/pipeline.html#from-pretrained).

A pipeline configuration is composed of several other [configuration classes](https://recognai.github.io/biome-text/v3.2.0/api/biome/text/configuration.html#biome-text-configuration), each one corresponding to one of the main components.

### Features

Let us first configure the features of our pipeline.
For our `word` feature we will use pretrained embeddings from [fasttext](https://fasttext.cc/docs/en/english-vectors.html), and our `char` feature will use the last hidden state of a [GRU](https://en.wikipedia.org/wiki/Gated_recurrent_unit) encoder to represent a word based on its characters.
Keep in mind that the `embedding_dim` parameter for the `word` feature must be equal to the dimensions of the pretrained embeddings!

::: tip Tip

If you do not provide any feature configurations, we will choose a very basic `word` feature by default.

:::

In [ ]:
word_feature = WordFeatures(
    embedding_dim=300,
    weights_file="https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip",
)

char_feature = CharFeatures(
    embedding_dim=32,
    encoder={
        "type": "gru",
        "bidirectional": True,
        "num_layers": 1,
        "hidden_size": 32,
    },
    dropout=0.1
)

features_config = FeaturesConfiguration(
    word=word_feature, 
    char=char_feature
)

### Encoder

Next we will configure our encoder that takes as input a sequence of embedded word vectors and returns a sequence of encoded word vectors.
For this encoding we will use another larger GRU:

In [ ]:
encoder_config = Seq2SeqEncoderConfiguration(
    type="gru",
    bidirectional=True,
    num_layers=1,
    hidden_size=128,
)

### Head

The final configuration belongs to our [TaskHead](https://recognai.github.io/biome-text/v3.2.0/api/biome/text/modules/heads/task_head.html#taskhead).
It reflects the task our problem belongs to and can be easily exchanged with other types of heads keeping the same features and encoder.

::: tip Tip

Exchanging the heads you can easily pretrain a model on a certain task, such as [language modelling](https://recognai.github.io/biome-text/v3.2.0/api/biome/text/modules/heads/language_modelling.html#languagemodelling), and use its pretrained features and encoder for training the model on another task.

:::

For our task we will use a [TokenClassification](https://recognai.github.io/biome-text/v3.2.0/api/biome/text/modules/heads/token_classification.html#tokenclassification) head that allows us to tag each token individually:

In [ ]:
head_config = TokenClassificationConfiguration(
    labels=list(labels),
    label_encoding="BIO",
    top_k=1,
    feedforward={
        "num_layers": 1,
        "hidden_dims": [128],
        "activations": ["relu"],
        "dropout": [0.1],
    },
)

### Pipeline

Now we can create a [PipelineConfiguration](https://recognai.github.io/biome-text/v3.2.0/api/biome/text/configuration.html#pipelineconfiguration) and finally initialize our [Pipeline](https://recognai.github.io/biome-text/v3.2.0/api/biome/text/pipeline.html#pipeline).

In [ ]:
pipeline_config = PipelineConfiguration(
    name="slot_filling_tutorial",
    features=features_config,
    encoder=encoder_config,
    head=head_config,
)

In [ ]:
pl = Pipeline.from_config(pipeline_config)

## Configure the vocabulary

The default behavior of *biome.text* is to add all tokens from the training data set to the pipeline's vocabulary. 
This is done automatically when training the pipeline for the first time.

For this tutorial we get rid of the rarest words by adding the `min_count` argument and set it to 2 for the word feature vocabulary.
Since we use pretrained word embeddings we will not only consider the training data, but also the validation data when creating the vocabulary by setting `include_valid_data=True`. 
For a complete list of available arguments see the [VocabularyConfiguration API](https://recognai.github.io/biome-text/v3.2.0/api/biome/text/configuration.html#vocabularyconfiguration).

In [ ]:
vocab_config = VocabularyConfiguration(min_count={"word": 2}, include_valid_data=True)

## Train your model

Now we have everything ready to start the training of our model:
- training data set
- pipeline

We will use the default configuration for our [`Trainer`](https://recognai.github.io/biome-text/v3.2.0/api/biome/text/trainer.html) that has sensible values and works alright for our experiment.
[This tutorial](https://recognai.github.io/biome-text/v3.2.0/documentation/tutorials/1-Training_a_text_classifier.html) provides more information about the `Trainer` and gives you an example how to configure it.

::: tip Tip

If you want to configure the trainer you can pass on a `TrainerConfiguration` instance to the `Trainer`s init. 
See the [TrainerConfiguration API](https://recognai.github.io/biome-text/v3.2.0/api/biome/text/configuration.html#trainerconfiguration) for a complete list of available configurations.

:::

In [ ]:
trainer = Trainer(
    pipeline=pl,
    train_dataset=train_ds,
    valid_dataset=valid_ds,
    vocab_config=vocab_config,
    trainer_config=None,
)

To start the training we simply call the `Trainer.fit()` method.

In [ ]:
trainer.fit()

Apart from estimating the generalization error with a validation data set, you might want to evaluate your model against a test data set to compare several models after optimizing their hyperparameters.
For this you can use the `Pipeline.evaluate()` method.

In [ ]:
pl.evaluate(test_ds)

The model above achieves an overall F1 score of around **0.95**, which is not bad when compared to [published results](https://nlpprogress.com/english/intent_detection_slot_filling.html) of the same data set.
You could continue the experiment changing the encoder to an LSTM network, try out a transformer architecture or fine tune the trainer.
But for now we will go on and make our first predictions with this trained model.

## Make your first predictions

Now that we trained our model we can go on to make our first predictions.

We provide the input expected by our `TaskHead` to the `Pipeline.predict()` method.
In our case it is a `TokenClassification` head that classifies a `text` input. 

You can either provide pretokenized tokens (list of strings) **or** a raw string to the `predict` method. In the first case, you should make sure that those tokens were tokenized the same way the training data was tokenized, in the latter case you should make sure that the pipeline uses the same tokenizer as was used for the training data.

The prediction of the `TokenClassification` head will always consist of a `tags` and `entities` key. Both keys will include the `top_k` most likely tag/entity sequences for the input, where `top_k` is a parameter specified in the `TokenClassificationConfiguration` before the training.

::: tip Tip

We can also load the trained pipeline from the training output. This is useful in case you trained the pipeline in some earlier session, and want to continue your work with the inference steps: 

```python
pl = Pipeline.from_pretrained("output/model.tar.gz")`
```

:::

### pretokenized input 

For pretokenized input, the `entities` key of the output holds dictionaries with the `start_token` id, `end_token` id and the label of the entity: 

In [ ]:
text = "Can you play biome text by backstreet recognais on Spotify ?".split()
prediction = pl.predict(text=text)

print("Predicted tags:\n", list(zip(text, prediction["tags"][0])))
print("Predicted entities:\n", prediction["entities"][0])

### string input

For a raw string input, the `entities` key of the output holds dictionaries with the `start_token` id, `end_token` id, `start` char id, `end` char id and the label of the entity:

In [ ]:
text = "Can you play biome text by backstreet recognais on Spotify ?"
prediction = pl.predict(text=text)

print("Predicted tags:\n", list(zip(text.split(), prediction["tags"][0])))
print("Predicted entities:\n", prediction["entities"][0])